## German Energy Data 

### Set up environment 

In [1]:
# set up environment 
import pandas as pd
import altair as alt

### Data import 

Our data was downloaded from [OPSD](https://data.open-power-system-data.org/time_series/2019-06-05).

In [2]:
# read in german energy data 
genergy = pd.read_csv('german_energy_data.csv', parse_dates=['datetime']) # parse timestamp column

### Data Exploration

To further explore our data, we can take a look at the shape, column names, and dtypes. Additionally, let's look at head and tail of our data. 

In [3]:
genergy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35064 entries, 0 to 35063
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype              
---  ------    --------------  -----              
 0   datetime  35064 non-null  datetime64[ns, UTC]
 1   load      35064 non-null  int64              
 2   solar     35064 non-null  int64              
 3   wind      35064 non-null  int64              
dtypes: datetime64[ns, UTC](1), int64(3)
memory usage: 1.1 MB


In [4]:
# head
genergy.head()

datetime   load  solar  wind
0 2014-12-31 23:00:00+00:00  41987      0  8644
1 2015-01-01 00:00:00+00:00  40665      0  8814
2 2015-01-01 01:00:00+00:00  39510      0  9054
3 2015-01-01 02:00:00+00:00  38537      0  9070
4 2015-01-01 03:00:00+00:00  38153      0  9164

In [5]:
# tail 
genergy.tail()

datetime   load  solar   wind
35059 2018-12-31 18:00:00+00:00  52829      0  16438
35060 2018-12-31 19:00:00+00:00  49667      0  17587
35061 2018-12-31 20:00:00+00:00  47369      0  18872
35062 2018-12-31 21:00:00+00:00  46890      0  20270
35063 2018-12-31 22:00:00+00:00  45079      0  21405

### Data Processing

From what we can see, our dataset contains a datapoint from 2014. However, if we look at the dtype of the column for datetime, we can see that the timezone is UTC, which is different from Germany's timezone. We can convert our data to Europe/Berlin timezone and see if this corrects it.

#### Convert timezone

In [6]:
# convert timezone to Europe/Berlin
genergy['datetime'] = genergy['datetime'].dt.tz_convert('Europe/Berlin')
genergy.head()

datetime   load  solar  wind
0 2015-01-01 00:00:00+01:00  41987      0  8644
1 2015-01-01 01:00:00+01:00  40665      0  8814
2 2015-01-01 02:00:00+01:00  39510      0  9054
3 2015-01-01 03:00:00+01:00  38537      0  9070
4 2015-01-01 04:00:00+01:00  38153      0  9164

Changing the timezone solved our problem! 

#### Set datetime as index 

Now, to make our indexing more quick and convenient, we can use our datetime as the dataframe: 

In [7]:
# change index to datetime
genergy = genergy.set_index('datetime') 

In [8]:
genergy.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 35064 entries, 2015-01-01 00:00:00+01:00 to 2018-12-31 23:00:00+01:00
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   load    35064 non-null  int64
 1   solar   35064 non-null  int64
 2   wind    35064 non-null  int64
dtypes: int64(3)
memory usage: 1.1 MB


In [9]:
# check if 2014 data point is gone 
genergy.index.year.unique()

Int64Index([2015, 2016, 2017, 2018], dtype='int64', name='datetime')

#### Aggregate time data 

Altair recommends up to 5,000 data points per chart; Therefore, we need to aggregate our data in order to reduce the resolution. We can do this using the resample function: 

In [10]:
# resample by days 
days = genergy.resample('D').sum()
days

load  solar    wind
datetime                                         
2015-01-01 00:00:00+01:00  1061334  17079  311300
2015-01-02 00:00:00+01:00  1267901   7758  601954
2015-01-03 00:00:00+01:00  1124883   7236  469165
2015-01-04 00:00:00+01:00  1072455  19984  390567
2015-01-05 00:00:00+01:00  1304357  26524  227372
...                            ...    ...     ...
2018-12-27 00:00:00+01:00  1224968  27663  286302
2018-12-28 00:00:00+01:00  1222846  27557  227879
2018-12-29 00:00:00+01:00  1208627  10390  511284
2018-12-30 00:00:00+01:00  1123319  12451  590481
2018-12-31 00:00:00+01:00  1119959   9329  230643

[1461 rows x 3 columns]

#### Wide-form to Long-form 

Now that we have resampled our data, we need to transform it into long-form data, which is what Altair works best with. We can do this by resetting the index followed by the melt function: 

In [11]:
# change to long-form
days = days.reset_index().melt('datetime', var_name='type') 
days

datetime  type    value
0    2015-01-01 00:00:00+01:00  load  1061334
1    2015-01-02 00:00:00+01:00  load  1267901
2    2015-01-03 00:00:00+01:00  load  1124883
3    2015-01-04 00:00:00+01:00  load  1072455
4    2015-01-05 00:00:00+01:00  load  1304357
...                        ...   ...      ...
4378 2018-12-27 00:00:00+01:00  wind   286302
4379 2018-12-28 00:00:00+01:00  wind   227879
4380 2018-12-29 00:00:00+01:00  wind   511284
4381 2018-12-30 00:00:00+01:00  wind   590481
4382 2018-12-31 00:00:00+01:00  wind   230643

[4383 rows x 3 columns]

### Data Visualization

Now that our data is in long-form, we can visualize it and see if there are any patterns. 

In [13]:
alt.Chart(days).mark_circle(size=15, opacity=0.75).encode(
    x = 'datetime', 
    y = 'value', 
    color = 'type', 
    tooltip = ['value']
).properties(
    width=800, 
    height=400, 
    title='Load, Solar, and Wind Power (2015-2018)'
)

alt.Chart(...)

Looking at our visualization, we can see a lot of patterns, but the patterns are not as distinct. To correct for this, we can create a line chart from the data:

In [14]:
alt.Chart(days).mark_line(strokeWidth=1).encode(
    x = 'datetime', 
    y = 'value', 
    color = 'type'
).properties(
    width=800, 
    height=400, 
    title='Load, Solar, and Wind Power (2015-2018)'
)

alt.Chart(...)

Above, we can see that the data contains a lot of fluctuations. Although we can see overall fluctuations over the course of months and years, there is still a lot of data. To reduce the amount of noise, we can change the sampling from days to months. 

In [120]:
# resampling to months, reset index, melt
months = genergy.resample('M').sum().reset_index().melt('datetime')

In [121]:
# to smooth out the line, we can use interpolation 
alt.Chart(months).mark_line(opacity=0.75, interpolate='basis').encode(
    x = 'datetime', 
    y = 'value', 
    color = 'variable'
).properties(
    width = 800, 
    height = 400, 
    title = 'Load, Solar, and Wind Power (2015-2018)'
)

alt.Chart(...)

Now, the chart above has lost too much detail due to the downsampling of the data as well as the interpolation. For example, at the end of each year we do not see the drop in load as pronounced as we do in the non-interpolated chart from before. 

To correct for this we can create a layered chart that contains both of the data. To accomplish this, we need to resample the data by daily sums and then generate the monthly averages from that:

In [122]:
# resampling 
days = genergy.resample('D').sum()
months = days.resample('M').mean()

In [142]:
# layered chart 
days_chart = alt.Chart(days.reset_index().melt('datetime')).mark_line(strokeWidth=1, opacity=0.25).encode(
    x = 'datetime',
    y = 'value',
    color = 'variable'
).properties(
    width = 800, 
    height = 400, 
    title = 'Load, Solar, and Wind Power (2015-2018)'
)

months_chart = alt.Chart(months.reset_index().melt('datetime')).mark_line(interpolate='basis').encode(
    x = 'datetime', 
    y = 'value', 
    color = 'variable'
).interactive() 

days_chart + months_chart 

alt.LayerChart(...)

After combining both charts, we can now see both the overall trends as well as the more detailed patterns within the data. However, theres a better way of smoothing out the data by using the rolling method. 

Our first paremeter determines the window size, which in our case is 60. Our values are centered and the variables are weighted using the 'triang' option, which makes values that are farther away contribute less. 

In [124]:
rolling = days.rolling(60, center=True, win_type='triang').mean()

In [129]:
rolling_chart = alt.Chart(rolling.reset_index().melt('datetime')).mark_line(strokeWidth=1.5, opacity=1).encode(
    x = 'datetime', 
    y = 'value', 
    color = 'variable'
).properties(
    width = 800, 
    height = 400, 
    title = 'Load, Solar, and Wind Power (2015-2018)' 
)

days_chart = alt.Chart(days.reset_index().melt('datetime')).mark_line(strokeWidth=1, opacity=0.25).encode(
    x = 'datetime',
    y = 'value',
    color = 'variable'
)

rolling_chart + days_chart

alt.LayerChart(...)

In the chart above, we can see that the rolling window data now has curves that better represent the dips within our data than the interpolate method. 

Now that we have a good sense of our overall data, we can create a more detailed view. Around 2018, we can see that there is a more pronounced peak for wind power. Let's take a closer look: 

In [132]:
# specify datetime 
detailed = genergy.loc['2018-01-01':'2018-01-07'].reset_index().melt('datetime')

In [133]:
alt.Chart(detailed).mark_line().encode(
    x = 'datetime', 
    y = 'value', 
    color = 'variable'
).properties(
    width = 800, 
    height = 400, 
    title = 'Load, Solar, and Wind Power (1/1/2018-1/7/2018)'
)

alt.Chart(...)

Instead of creating multiple charts with different views, we can create a high-level overview with functionality for detailed close-ups. 

In [134]:
# resample data 
weeks = genergy.resample('W').sum()
days = genergy.resample('D').sum() 

In [137]:
brush = alt.selection(type='interval', encodings=['x'])

upper = alt.Chart(weeks.reset_index().melt('datetime')).mark_area(interpolate='basis').encode(
    x = alt.X('datetime', axis=None), 
    y = alt.Y('value', axis=None), 
    color = 'variable'
).properties(
    width = 800, 
    height = 50, 
    title = 'Load, Solar, and Wind Power (2015-2018)'
).add_selection(brush)

lower = alt.Chart(days.reset_index().melt('datetime')).mark_line(strokeWidth=1).encode(
    x = alt.X('datetime', scale=alt.Scale(domain=brush)), 
    y = 'value', 
    color = 'variable'
).properties(width=800, height=400)

upper & lower

alt.VConcatChart(...)

In the above chart, we can now select a specific timeseries on the above area chart and it will be reflected on the line chart.

Aside from more detailed views from our data, we can also use the data we already have to create views of other facet's of the data such as renewable energy over the years. 

In [140]:
# resample data by years 
years = genergy.resample('Y').sum() 

# create new column 
years['renewable'] = years['solar'] + years['wind']

# drop columns 
renewable = years.drop(['load', 'solar', 'wind'], axis=1)

# create year column 
renewable['year'] = years.index.year

# reset index 
renewable = renewable.reset_index(drop=True) 

In [141]:
renewable

renewable  year
0  112376119  2015
1  111564166  2016
2  138533852  2017
3  149809554  2018

Now, we can create a chart to view our renewable energy use throughout the years. 

In [145]:
alt.Chart(renewable).mark_bar(width=30, fill='green').encode(
    x = 'year:O', 
    y = 'renewable'
).properties(
    width = 300, 
    height = 300, 
    title = 'Renewable Energy Use (2015-2018)'
)

alt.Chart(...)

As we can see from our above chart, over the years, renewable energy use has increased. 